<a href="https://colab.research.google.com/github/keshvi-srivastava/star-wars-dialogue-generation/blob/main/text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
import time
import re
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
path_to_file = '/content/sample_data/script_data.txt'

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 179673 characters


In [ ]:

# Taking a look at the text
text[:300]

"threepio: Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This is madness!\nthreepio: We're doomed!\nthreepio: There'll be no escape for the Princess this time.\nthreepio: What's that?\nthreepio: I should have known better than to trust the logic of a half-sized th"

Python for NLP Deep Learning text gen with keras


In [ ]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence.lower()

In [ ]:
text = preprocess_text(text)

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

27 unique characters


In [ ]:
print(vocab)

[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
text_words = (word_tokenize(text))
n_words = len(text_words)
unique_words = len(set(text_words))

print('Total Words: %d' % n_words)
print('Unique Words: %d' % unique_words)

Total Words: 31032
Unique Words: 3503


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=3437)
tokenizer.fit_on_texts(text_words)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
word_2_index = tokenizer.word_index

In [ ]:
input_sequence = []
output_words = []
input_seq_length = 100

for i in range(0, n_words - input_seq_length , 1):
    in_seq = text_words[i:i + input_seq_length]
    out_seq = text_words[i + input_seq_length]
    input_sequence.append([word_2_index[word] for word in in_seq])
    output_words.append(word_2_index[out_seq])

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.utils import to_categorical
from random import randint

In [ ]:
X = np.reshape(input_sequence, (len(input_sequence), input_seq_length, 1))
X = X / float(vocab_size)

y = to_categorical(output_words)

In [ ]:
model = Sequential()
model.add(LSTM(800, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(800, return_sequences=True))
model.add(LSTM(800))
model.add(Dense(y.shape[1], activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 800)          2566400   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 800)          5123200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 800)               5123200   
_________________________________________________________________
dense (Dense)                (None, 3504)              2806704   
Total params: 15,619,504
Trainable params: 15,619,504
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, batch_size=64, epochs=10, verbose=1)

Epoch 1/10
484/484 [==============================] - 87s 112ms/step - loss: 6.5413
Epoch 2/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1911
Epoch 3/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1741
Epoch 4/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1726
Epoch 5/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1815
Epoch 6/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1708
Epoch 7/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1763
Epoch 8/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1699
Epoch 9/10
484/484 [==============================] - 55s 113ms/step - loss: 6.1594
Epoch 10/10
484/484 [==============================] - 55s 114ms/step - loss: 6.1718


In [ ]:
random_seq_index = np.random.randint(0, len(input_sequence)-1)
random_seq = input_sequence[random_seq_index]

index_2_word = dict(map(reversed, word_2_index.items()))

word_sequence = [index_2_word[value] for value in random_seq]

print(' '.join(word_sequence))

the hyperdrive motivator has been damaged it impossible to go to light speed han we re in trouble han horizontal boosters han alluvial dampers well that not it han bring me the hydrospanners han don know how we re going to get out of this one han oww chewie han that was no laser blast something hit us leia han get up here han come on chewie leia asteroids han oh no chewie set two seven one leia what are you doing you re not actually going into an asteroid field han they be crazy to follow us wouldn they


In [ ]:
for i in range(100):
    int_sample = np.reshape(random_seq, (1, len(random_seq), 1))
    int_sample = int_sample / float(vocab_size)

    predicted_word_index = model.predict(int_sample, verbose=0)

    predicted_word_id = np.argmax(predicted_word_index)
    seq_in = [index_2_word[index] for index in random_seq]

    word_sequence.append(index_2_word[ predicted_word_id])

    random_seq.append(predicted_word_id)
    random_seq = random_seq[1:len(random_seq)]

In [ ]:
final_output = ""
for word in word_sequence:
    final_output = final_output + " " + word

print(final_output)

 the hyperdrive motivator has been damaged it impossible to go to light speed han we re in trouble han horizontal boosters han alluvial dampers well that not it han bring me the hydrospanners han don know how we re going to get out of this one han oww chewie han that was no laser blast something hit us leia han get up here han come on chewie leia asteroids han oh no chewie set two seven one leia what are you doing you re not actually going into an asteroid field han they be crazy to follow us wouldn they you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you you


Change the hyper parameters, including the size and number of LSTM layers and number of epochs to see if you get better results.
Try to remove the stop words like is, am, are from training set to generate words other than stop words in the test set (although this will depend on the type of application).
Create a character-level text generation model that predicts the next N characters.


HP Text Generator example


In [ ]:
# Creating a mapping from unique characters to indices
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)

text_as_int = np.array([char2index[c] for c in text])

print(text_as_int)

[63 51 61 ... 63  2  0]


In [ ]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} -- characters mapped to int -- > {}'.format(repr(text[:13]), text_as_int[:13]))

'threepio: Did' -- characters mapped to int -- > [63 51 61 48 48 59 52 58 16  1 22 52 47]


In [ ]:

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(index2char[i.numpy()])

t
h
r
e
e


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(index2char[item.numpy()])))

"threepio: Did you hear that?  They've shut down the main reactor.  We'll be destroyed for sure.  This"
" is madness!\nthreepio: We're doomed!\nthreepio: There'll be no escape for the Princess this time.\nthre"
"epio: What's that?\nthreepio: I should have known better than to trust the logic of a half-sized therm"
'ocapsulary dehousing assister...\nthreepio: Artoo! Artoo-Detoo, where are you?\nthreepio: At last!  Whe'
"re have you been?\nthreepio: They're heading in this direction. What are we going to do?  We'll be sen"


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 300 #256

# Number of RNN units
rnn_units1 = 512 #1024
rnn_units2 = 256
rnn_units=[rnn_units1, rnn_units2]
print(vocab_size)

74


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [ ]:
model = build_model(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 300)           22200     
_________________________________________________________________
gru (GRU)                    (64, None, 512)           1250304   
_________________________________________________________________
gru_1 (GRU)                  (64, None, 256)           591360    
_________________________________________________________________
dense (Dense)                (64, None, 74)            19018     
Total params: 1,882,882
Trainable params: 1,882,882
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
EPOCHS=50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
27/27 [==============================] - 48s 2s/step - loss: 3.8575 - accuracy: 0.1231
Epoch 2/50
27/27 [==============================] - 45s 2s/step - loss: 3.1474 - accuracy: 0.1827
Epoch 3/50
27/27 [==============================] - 43s 2s/step - loss: 2.7789 - accuracy: 0.2546
Epoch 4/50
27/27 [==============================] - 43s 2s/step - loss: 2.3850 - accuracy: 0.3393
Epoch 5/50
27/27 [==============================] - 43s 2s/step - loss: 2.1846 - accuracy: 0.3827
Epoch 6/50
27/27 [==============================] - 43s 2s/step - loss: 2.0512 - accuracy: 0.4145
Epoch 7/50
27/27 [==============================] - 43s 2s/step - loss: 1.9402 - accuracy: 0.4400
Epoch 8/50
27/27 [==============================] - 44s 2s/step - loss: 1.8477 - accuracy: 0.4626
Epoch 9/50
27/27 [==============================] - 43s 2s/step - loss: 1.7616 - accuracy: 0.4867
Epoch 10/50
27/27 [==============================] - 44s 2s/step - loss: 1.7022 - accuracy: 0.5029
Epoch 11/50
27/27 [

In [ ]:
latest_check= tf.train.latest_checkpoint(checkpoint_dir)

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(latest_check)

model.build(tf.TensorShape([1, None]))

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 300)            22200     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 512)            1250304   
_________________________________________________________________
gru_3 (GRU)                  (1, None, 256)            591360    
_________________________________________________________________
dense_1 (Dense)              (1, None, 74)             19018     
Total params: 1,882,882
Trainable params: 1,882,882
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low results in more predictable text.
  # Higher results in more surprising text.
  # Experiment to find the best setting.
  scaling = 0.5 #1

  # batch size == 1
  
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / scaling
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden stte
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(index2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"yoda:"))

yoda: Arrogant in a planets havedisappear.  What's he saying?
threepio: Well, I should have known on the outskirts and bring usioned to the ship. I'm staying your bunderesting a dream.
vader: I like two or three, but I'm not sure this is the father, isn't he?
ben: (continuing) What's this asteroid field so that we meet on the Jedi the end of your: Artoo.
vader: (subtitled) Chess the galaxy as fast assion. The Jedi Council is some rooking for.
vader: Here we go.
ben: I'm trying to stop! Whoa!
vader: (Cont'd) I'm trying to take her. I don't understand. Stay on the Dook. I will do what you've done . . . what you were you all right?
vader: It's not a ship.  I'm in it for the making . . .
vader: I'm going to save Obi-Wan. I sense something rather way.
threepio: We're doomed.
threepio: I think I know what you're talking about his mining to go.
vader: Yes, Anakin.
vader: I don't know where they don't trust you.
padme: Anakin, this is my droids are wanted of the Force, that incomplete will see